### Transit ridership panel data models

Script to estimate the models of MUNI bus and rail ridership at a TAZ level for the purpose of determining the effect of TNCs on transit ridership in San Francisco. 

gde 3.28.2020

In [2]:
import pandas as pd
import numpy as np

from linearmodels.panel import PanelOLS

pd.set_option('display.max_columns', 500)
pd.set_option('display.max_rows', 20)

%matplotlib inline

In [3]:
# read the data and structure the data
data = pd.read_csv('data/PanelEstimationFile.csv', thousands = ',')


In [4]:
# calculate dervied fields 

# setting flags
data['BUS'] = np.where(data['MODE']=='BUS', 1, 0)
data['RAIL'] = np.where(data['MODE']=='RAIL', 1, 0)
data['YEAR_2015'] = np.where(data['YEAR']==2015, 1, 0)

# segmenting bus vs rail coefficients
data['TRIP_STOPS_DELIVERED_BUS'] = data['TRIP_STOPS_DELIVERED']*data['BUS']
data['TRIP_STOPS_DELIVERED_RAIL'] = data['TRIP_STOPS_DELIVERED']*data['RAIL']

data['AVG_TNC_SMOOTH_BUS'] = data['AVG_TNC_SMOOTH'] * data['BUS']
data['AVG_TNC_SMOOTH_RAIL'] = data['AVG_TNC_SMOOTH'] * data['RAIL']

data['BUS_TRIP_STOPS_COMPETING_BUS'] = data['BUS_TRIP_STOPS_COMPETING'] * data['BUS']

# combining BART and Caltrain
data['BART_CT_AVG_RIDE_SMOOTH'] = data['BART_AVG_RIDE_SMOOTH'] + data['CALTRAIN_AVG_RIDE_SMOOTH']


In [5]:
# Set the index
data = data.set_index(['ID', 'YEAR'])
data.head()

Unnamed: 0  SFTAZ MODE       TOD2  DIST    DISTNAME  \
ID              YEAR                                                        
1_BUS_0300-0859 2010           0      1  BUS  0300-0859     9  'Bayshore'   
                2015           1      1  BUS  0300-0859     9  'Bayshore'   
1_BUS_0900-1559 2010           2      1  BUS  0900-1559     9  'Bayshore'   
                2015           3      1  BUS  0900-1559     9  'Bayshore'   
1_BUS_1600-1859 2010           4      1  BUS  1600-1859     9  'Bayshore'   

                       AVG_RIDE  ROUTES  STOPS  TRIP_STOPS  \
ID              YEAR                                         
1_BUS_0300-0859 2010  11.503687     2.0    3.0        36.0   
                2015  33.240093     3.0    3.0        50.0   
1_BUS_0900-1559 2010  50.245435     2.0    3.0        84.0   
                2015  56.483018     2.0    3.0       119.0   
1_BUS_1600-1859 2010  31.448123     2.0    3.0        36.0   

                      TRIP_STOPS_DELIVERED  RAPID_SHARE  \
ID              YEAR                                      
1_BUS_0300-0859 2010                34.884     0.000000   
                2015                49.800     0.800000   
1_BUS_0900-1559 2010                81.396     0.000000   
                2015               118.524     0.890756   
1_BUS_1600-1859 2010                34.884     0.000000   

                      SERVMILES_S_DELIVERED  RUNSPEED   ONTIME5  \
ID              YEAR                                              
1_BUS_0300-0859 2010               3.162816     12.26  0.594530   
                2015               4.519848      5.47  0.448981   
1_BUS_0900-1559 2010               7.366338     12.81  0.572105   
                2015              10.746840      5.76  0.505889   
1_BUS_1600-1859 2010               3.157002     11.63  0.408982   

                      BUS_TRIP_STOPS_COMPETING  HHLDS   POP  TOTALEMP  EMPRES  \
ID              YEAR                                                            
1_BUS_0300-0859 2010                 14.992544    306  1303        50     506   
                2015                 18.880575    317  1353        69     546   
1_BUS_0900-1559 2010                 31.462386    306  1303        50     506   
                2015                 41.739716    317  1353        69     546   
1_BUS_1600-1859 2010                 13.744592    306  1303        50     506   

                      ACCESS_30  HHLDS_0_VEH_SHARE  BART_AVG_RIDE  \
ID              YEAR                                                
1_BUS_0300-0859 2010   302980.0           0.227414            0.0   
                2015   351207.0           0.169343            0.0   
1_BUS_0900-1559 2010   219577.0           0.227414            0.0   
                2015   189411.0           0.169343            0.0   
1_BUS_1600-1859 2010   124505.0           0.227414            0.0   

                      CALTRAIN_AVG_RIDE  AVG_TNC  HHLDS_SMOOTH  POP_SMOOTH  \
ID              YEAR                                                         
1_BUS_0300-0859 2010                0.0    0.000     71.422712  297.242800   
                2015                0.0    3.395     74.882905  311.916768   
1_BUS_0900-1559 2010                0.0    0.000     71.422712  297.242800   
                2015                0.0    3.725     74.882905  311.916768   
1_BUS_1600-1859 2010                0.0    0.000     71.422712  297.242800   

                      TOTALEMP_SMOOTH  EMPRES_SMOOTH  \
ID              YEAR                                   
1_BUS_0300-0859 2010        12.223620     115.311125   
                2015        17.408276     125.629692   
1_BUS_0900-1559 2010        12.223620     115.311125   
                2015        17.408276     125.629692   
1_BUS_1600-1859 2010        12.223620     115.311125   

                      HHLDS_0_VEH_SHARE_SMOOTH  BART_AVG_RIDE_SMOOTH  \
ID              YEAR                                                   
1_BUS_0300-0859 2010       

In [6]:
# match Greg's combined TAZ model

mod = PanelOLS.from_formula("np.log(1+AVG_RIDE) \
                            ~ np.log(1+ACCESS_30) \
                            + np.log(1+ROUTES) \
                            + np.log(1+TRIP_STOPS_DELIVERED_BUS) \
                            + np.log(1+TRIP_STOPS_DELIVERED_RAIL) \
                            + ONTIME5 \
                            + np.log(1+BART_CT_AVG_RIDE_SMOOTH) \
                            + np.log(1+BUS_TRIP_STOPS_COMPETING_BUS) \
                            + np.log(1+AVG_TNC_SMOOTH_BUS) \
                            + np.log(1+AVG_TNC_SMOOTH_RAIL) \
                            + YEAR_2015 \
                            + EntityEffects",
             data = data)

res = mod.fit()
print(res)

                           PanelOLS Estimation Summary                            
Dep. Variable:     np.log(1 + AVG_RIDE)   R-squared:                        0.3234
Estimator:                     PanelOLS   R-squared (Between):              0.8894
No. Observations:                  7358   R-squared (Within):               0.3234
Date:                  Thu, Apr 23 2020   R-squared (Overall):              0.8879
Time:                          11:17:02   Log-likelihood                    1897.1
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      175.41
Entities:                          3679   P-value                           0.0000
Avg Obs:                         2.0000   Distribution:                 F(10,3669)
Min Obs:                         2.0000                                           
Max Obs:                         2.0000   F-statistic (robust):             175.41
    